<a href="https://colab.research.google.com/github/papichoolo/Soybeans/blob/main/SoyNet_Beans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mounting the dataset from Google Drive and importing libraries

In [ ]:
import numpy as np
import tensorflow as tf

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir('/content/drive/MyDrive/Camera Clicks_256_256/train')

print (dataset_path)  #what kinds of classes are in this dataset

print("Types of classes labels found: ", len(dataset_path))

['Disease_Preprocessing data', 'Healthy_Preprocessing data']
Types of classes labels found:  2


In [ ]:
class_labels = []

for item in dataset_path:
 # Get all the file names
 all_classes = os.listdir('/content/drive/MyDrive/Camera Clicks_256_256/train' + '/' +item)
 print(all_classes)

 # Add them to the list
 for room in all_classes:
    class_labels.append((item, str('dataset_path' + '/' +item) + '/' + room))

['Soy (1060).JPG', 'Soy (106).JPG', 'Soy (1061).JPG', 'Soy (1062).JPG', 'Soy (1064).JPG', 'Soy (1063).JPG', 'Soy (1066).JPG', 'Soy (1065).JPG', 'Soy (1068).JPG', 'Soy (1067).JPG', 'Soy (1069).JPG', 'Soy (1070).JPG', 'Soy (1071).JPG', 'Soy (1072).JPG', 'Soy (1074).JPG', 'Soy (1073).JPG', 'Soy (1075).JPG', 'Soy (1076).JPG', 'Soy (1078).JPG', 'Soy (1077).JPG', 'Soy (1080).JPG', 'Soy (1079).JPG', 'Soy (1082).JPG', 'Soy (1081).JPG', 'Soy (1083).JPG', 'Soy (1084).JPG', 'Soy (1085).JPG', 'Soy (1086).JPG', 'Soy (1087).JPG', 'Soy (1088).JPG', 'Soy (1089).JPG', 'Soy (109).JPG', 'Soy (1090).JPG', 'Soy (1092).JPG', 'Soy (1091).JPG', 'Soy (1094).JPG', 'Soy (1093).JPG', 'Soy (1095).JPG', 'Soy (1096).JPG', 'Soy (1097).JPG', 'Soy (1098).JPG', 'Soy (1099).JPG', 'Soy (110).JPG', 'Soy (11).JPG', 'Soy (1101).JPG', 'Soy (1100).JPG', 'Soy (1102).JPG', 'Soy (1104).JPG', 'Soy (1105).JPG', 'Soy (1106).JPG', 'Soy (1107).JPG', 'Soy (1108).JPG', 'Soy (1109).JPG', 'Soy (111).JPG', 'Soy (1110).JPG', 'Soy (1111).JPG

Creating a dataframe of Images and labels

In [ ]:
df = pd.DataFrame(data=class_labels, columns=['Labels', 'image'])
df

,Labels,image
0,Disease_Preprocessing data,dataset_path/Disease_Preprocessing data/Soy (1...
1,Disease_Preprocessing data,dataset_path/Disease_Preprocessing data/Soy (1...
2,Disease_Preprocessing data,dataset_path/Disease_Preprocessing data/Soy (1...
3,Disease_Preprocessing data,dataset_path/Disease_Preprocessing data/Soy (1...
4,Disease_Preprocessing data,dataset_path/Disease_Preprocessing data/Soy (1...
...,...,...
1189,Healthy_Preprocessing data,dataset_path/Healthy_Preprocessing data/Soy (9...
1190,Healthy_Preprocessing data,dataset_path/Healthy_Preprocessing data/Soy (7...
1191,Healthy_Preprocessing data,dataset_path/Healthy_Preprocessing data/Soy (9...
1192,Healthy_Preprocessing data,dataset_path/Healthy_Preprocessing data/Soy (5...


In [ ]:
print("Total number of images in the dataset: ", len(df))

label_count = df['Labels'].value_counts()
print(label_count)

Total number of images in the dataset:  1194
Disease_Preprocessing data    1055
Healthy_Preprocessing data     139
Name: Labels, dtype: int64


Reading the Image and Resizing it to 224 x 224

In [ ]:
import cv2
path = '/content/drive/MyDrive/Camera Clicks_256_256/train/'
dataset_path = os.listdir('/content/drive/MyDrive/Camera Clicks_256_256/train/')

im_size = 224

images = []
labels = []

for i in dataset_path:
    data_path = path + str(i)
    filenames = [i for i in os.listdir(data_path) ]

    for f in filenames:
        img = cv2.imread(data_path + '/' + f)
        img = cv2.resize(img, (im_size, im_size))
        images.append(img)
        labels.append(i)

In [ ]:
images = np.array(images)

images = images.astype('float32') / 255.0
images.shape

(1194, 224, 224, 3)

Encoding labels

In [ ]:
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
y=df['Labels'].values
print(y)

y_labelencoder = LabelEncoder ()
y = y_labelencoder.fit_transform (y)
print (y)

['Disease_Preprocessing data' 'Disease_Preprocessing data'
 'Disease_Preprocessing data' ... 'Healthy_Preprocessing data'
 'Healthy_Preprocessing data' 'Healthy_Preprocessing data']
[0 0 0 ... 1 1 1]


In [ ]:
y=y.reshape(-1,1)

from sklearn.compose import ColumnTransformer
ct = ColumnTransformer([('my_ohe', OneHotEncoder(), [0])], remainder='passthrough')
Y = ct.fit_transform(y) #.toarray()
print(Y[:5])
print(Y[35:])

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


images, y = shuffle(images, Y, random_state=1)


train_x, test_x, train_y, test_y = train_test_split(images, y, test_size=0.05, random_state=415)

#inpect the shape of the training and testing.
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(1134, 224, 224, 3)
(1134, 2)
(60, 224, 224, 3)
(60, 2)


Using EfficientNetB0

Using a Learning Rate scheduler to create a custom Learning rate

In [ ]:
from keras.callbacks import LearningRateScheduler
def lr_schedule(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * 0.9


Trying to implement SGD Function

In [ ]:
import keras
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)


WandB Metrics uploaded

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0

NUM_CLASSES = 2
IMG_SIZE = 224
size = (IMG_SIZE, IMG_SIZE)

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

# Using EfficientNetB0 model without top classification layer
base_model = EfficientNetB0(weights=None, include_top=False, input_tensor=inputs, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Add your custom classification layers on top of the base model
x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)

# Combine the base model with custom classification layers
model_ = tf.keras.Model(inputs=base_model.input, outputs=outputs)

# Load pre-trained weights
#model.load_weights("/content/soybean_effecientnet.keras")

# Compile the model with appropriate optimizer, loss function, and metrics for your new dataset
model_.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=["accuracy","F1Score","Precision","Recall"])

# Print model summary
model_.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 224, 224, 3)          0         ['input_1[0][0]']             
                                                                                                  
 normalization (Normalizati  (None, 224, 224, 3)          7         ['rescaling[0][0]']           
 on)                                                                                              
                                                                                                  
 stem_conv_pad (ZeroPadding  (None, 225, 225, 3)          0         ['normalization[0][0]']   

In [ ]:
scheduler= LearningRateScheduler(lr_schedule)

hist = model_.fit(train_x, train_y, validation_split=0.2, epochs=50, verbose=1, callbacks=[scheduler])


Epoch 1/50
29/29 [==============================] - 44s 429ms/step - loss: 1.3456 - accuracy: 0.8346 - f1_score: 0.4972 - precision: 0.8346 - recall: 0.8346 - val_loss: 226923.6250 - val_accuracy: 0.9075 - val_f1_score: 0.4758 - val_precision: 0.9075 - val_recall: 0.9075 - lr: 0.0100
Epoch 2/50
29/29 [==============================] - 8s 277ms/step - loss: 0.6739 - accuracy: 0.8776 - f1_score: 0.4762 - precision: 0.8776 - recall: 0.8776 - val_loss: 0.3088 - val_accuracy: 0.9075 - val_f1_score: 0.4758 - val_precision: 0.9075 - val_recall: 0.9075 - lr: 0.0100
Epoch 3/50
29/29 [==============================] - 7s 258ms/step - loss: 0.6498 - accuracy: 0.8776 - f1_score: 0.4674 - precision: 0.8776 - recall: 0.8776 - val_loss: 0.3834 - val_accuracy: 0.9075 - val_f1_score: 0.4758 - val_precision: 0.9075 - val_recall: 0.9075 - lr: 0.0100
Epoch 4/50
29/29 [==============================] - 8s 270ms/step - loss: 0.5696 - accuracy: 0.8765 - f1_score: 0.4843 - precision: 0.8765 - recall: 0.8765 -

accuracy of 95.59% in only 5 epochs?? yep thats learning rate scheduling :)))

In [ ]:
preds = model_.evaluate(test_x, test_y)
test_loss=preds[0]
test_accuracy=preds[1]


2/2 [==============================] - 1s 1s/step - loss: 1.7338 - accuracy: 0.8500 - f1_score: 0.4595 - precision: 0.8500 - recall: 0.8500


Save Model

In [ ]:
model_.save('soybean_effecientnet.keras')

### Mobile Data Loaded for Validation (wasteland alert!!!!)

Loading Mobile Image Data

TAKING UNSEEN SOYABEAN LEAF IMAGE


It predicts diseased leaf, which is correct.

In [ ]:
f1= hist.history['f1_score']
f1_scores = []
prec_list=[]
rec_list=[]
for pair in f1:
    precision = pair[0]
    recall = pair[1]
    f1_ = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
    f1_scores.append(f1_)
    prec_list.append(precision)
    rec_list.append(recall)

# Convert the list of F1 scores to a NumPy array
f1_scores_array = np.array(f1_scores)
prec_list= np.array(prec_list)
rec_list= np.array(rec_list)
print(f1_scores_array)
print(prec_list)
print(rec_list)

[0.15607582 0.03474043 0.         0.0665101  0.09883758 0.
 0.         0.         0.         0.         0.         0.
 0.06940338 0.06384832 0.         0.09722904 0.03567118 0.28288644
 0.14957063 0.         0.0665101  0.17933768 0.20295061 0.23323876
 0.32939688 0.38798618 0.46152371 0.44026811 0.54890355 0.17799615
 0.5063058  0.56082924 0.48229522 0.47729269 0.75983719 0.70100616
 0.754198   0.72895786 0.77584226 0.78993218 0.79437566 0.81266831
 0.79937267 0.83869032 0.71188426 0.79756081 0.82156242 0.77125209
 0.82926328 0.82327865]
[0.9090909  0.93474424 0.9348209  0.93404    0.9358446  0.9360704
 0.935446   0.9360704  0.9360704  0.9360704  0.9360704  0.9360704
 0.9371697  0.9308919  0.9360704  0.93459046 0.9366197  0.9346017
 0.9331756  0.9360704  0.93404    0.93561727 0.9355411  0.93031573
 0.9330944  0.92947555 0.9315567  0.9376499  0.9407497  0.9349882
 0.93381906 0.9379562  0.9373493  0.9387019  0.9491526  0.94482327
 0.95214415 0.9477612  0.95232123 0.95625    0.9568481  0.

In [ ]:
import plotly.graph_objs as go
import numpy as np
epochs=list(range(1,51))

trace_accuracy = go.Scatter(x=epochs, y=hist.history['accuracy'], mode='lines', name='Accuracy')
trace_precision = go.Scatter(x=epochs, y=prec_list, mode='lines', name='Precision')
trace_recall = go.Scatter(x=epochs, y=rec_list, mode='lines', name='Recall')
trace_f1_score = go.Scatter(x=epochs, y=f1_scores_array, mode='lines', name='F1 Score')

layout = go.Layout(
    title='Metrics Across Epochs',
    xaxis=dict(title='Epochs'),
    yaxis=dict(title='Value'),
)

# Combining traces and layout
data = [trace_accuracy, trace_precision, trace_recall, trace_f1_score]
fig = go.Figure(data=data, layout=layout)

# Displaying the plot
fig.show()

Folder mgmt


### Transfer Learning

In [ ]:
import numpy as np
import tensorflow as tf

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir('/content/drive/MyDrive/Camera Clicks_256_256/val')

print (dataset_path)  #what kinds of classes are in this dataset

print("Types of classes labels found: ", len(dataset_path))

['healthy', 'diseased']
Types of classes labels found:  2


In [ ]:
class_labels = []

for item in dataset_path:
 # Get all the file names
 all_classes = os.listdir('/content/drive/MyDrive/Camera Clicks_256_256/val' + '/' +item)
 print(all_classes)

 # Add them to the list
 for room in all_classes:
    class_labels.append((item, str('dataset_path' + '/' +item) + '/' + room))

['soy (144).jpg', 'soy (158).jpg', 'soy (160).jpg', 'soy (17).jpg', 'soy (11).jpg', 'soy (31).jpg', 'soy (54).jpg', 'soy (53).jpg', 'soy (33).jpg', 'soy (73).jpg', 'soy (71).jpg', 'soy (57).jpg', 'soy (138).jpg', 'soy (111).jpg', 'soy (93).jpg', 'soy (94).jpg', 'soy (113).jpg', 'soy (133).jpg', 'soy (139).jpg', 'soy (140).jpg', 'soy (134).jpg']
['soy (142).jpg', 'soy (143).jpg', 'soy (145).jpg', 'soy (146).jpg', 'soy (147).jpg', 'soy (148).jpg', 'soy (149).jpg', 'soy (150).jpg', 'soy (151).jpg', 'soy (152).jpg', 'soy (153).jpg', 'soy (154).jpg', 'soy (155).jpg', 'soy (156).jpg', 'soy (157).jpg', 'soy (159).jpg', 'soy (161).jpg', 'soy (162).jpg', 'soy (163).jpg', 'soy (164).jpg', 'soy (165).jpg', 'soy (166).jpg', 'soy (167).jpg', 'soy (168).jpg', 'soy (169).jpg', 'soy (1).jpg', 'soy (2).jpg', 'soy (3).jpg', 'soy (4).jpg', 'soy (5).jpg', 'soy (6).jpg', 'soy (7).jpg', 'soy (8).jpg', 'soy (9).jpg', 'soy (10).jpg', 'soy (141).jpg', 'soy (20).jpg', 'soy (44).jpg', 'soy (38).jpg', 'soy (50).j

In [ ]:
df1 = pd.DataFrame(data=class_labels, columns=['Labels', 'image'])
df1

,Labels,image
0,healthy,dataset_path/healthy/soy (144).jpg
1,healthy,dataset_path/healthy/soy (158).jpg
2,healthy,dataset_path/healthy/soy (160).jpg
3,healthy,dataset_path/healthy/soy (17).jpg
4,healthy,dataset_path/healthy/soy (11).jpg
...,...,...
164,diseased,dataset_path/diseased/soy (16).jpg
165,diseased,dataset_path/diseased/soy (13).jpg
166,diseased,dataset_path/diseased/soy (14).jpg
167,diseased,dataset_path/diseased/soy (12).jpg


In [ ]:
print("Total number of images in the dataset: ", len(df1))

label_count = df1['Labels'].value_counts()
print(label_count)

Total number of images in the dataset:  169
diseased    148
healthy      21
Name: Labels, dtype: int64


In [ ]:
import cv2
path = '/content/drive/MyDrive/Camera Clicks_256_256/val/'
dataset_path = os.listdir('/content/drive/MyDrive/Camera Clicks_256_256/val/')

im_size = 224

images = []
labels = []

for i in dataset_path:
    data_path = path + str(i)
    filenames = [i for i in os.listdir(data_path) ]

    for f in filenames:
        img = cv2.imread(data_path + '/' + f)
        img = cv2.resize(img, (im_size, im_size))
        images.append(img)
        labels.append(i)

In [ ]:
images = np.array(images)

images = images.astype('float32') / 255.0
images.shape

(169, 224, 224, 3)

In [ ]:
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
y=df1['Labels'].values
print(y)

y_labelencoder = LabelEncoder ()
y = y_labelencoder.fit_transform (y)
print (y)

['healthy' 'healthy' 'healthy' 'healthy' 'healthy' 'healthy' 'healthy'
 'healthy' 'healthy' 'healthy' 'healthy' 'healthy' 'healthy' 'healthy'
 'healthy' 'healthy' 'healthy' 'healthy' 'healthy' 'healthy' 'healthy'
 'diseased' 'diseased' 'diseased' 'diseased' 'diseased' 'diseased'
 'diseased' 'diseased' 'diseased' 'diseased' 'diseased' 'diseased'
 'diseased' 'diseased' 'diseased' 'diseased' 'diseased' 'diseased'
 'diseased' 'diseased' 'diseased' 'diseased' 'diseased' 'diseased'
 'diseased' 'diseased' 'diseased' 'diseased' 'diseased' 'diseased'
 'diseased' 'diseased' 'diseased' 'diseased' 'diseased' 'diseased'
 'diseased' 'diseased' 'diseased' 'diseased' 'diseased' 'diseased'
 'diseased' 'diseased' 'diseased' 'diseased' 'diseased' 'diseased'
 'diseased' 'diseased' 'diseased' 'diseased' 'diseased' 'diseased'
 'diseased' 'diseased' 'diseased' 'diseased' 'diseased' 'diseased'
 'diseased' 'diseased' 'diseased' 'diseased' 'diseased' 'diseased'
 'diseased' 'diseased' 'diseased' 'diseased' 'dise

In [ ]:
y=y.reshape(-1,1)

from sklearn.compose import ColumnTransformer
ct = ColumnTransformer([('my_ohe', OneHotEncoder(), [0])], remainder='passthrough')
Y = ct.fit_transform(y) #.toarray()
print(Y[:5])
print(Y[35:])

In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


images, y = shuffle(images, Y, random_state=1)


train_x, test_x, train_y, test_y = train_test_split(images, y, test_size=0.05, random_state=415)

#inpect the shape of the training and testing.
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(160, 224, 224, 3)
(160, 2)
(9, 224, 224, 3)
(9, 2)


In [ ]:
train_y.shape

(160, 2)

In [ ]:
NUM_CLASSES = 2
IMG_SIZE = 224
size = (IMG_SIZE, IMG_SIZE)

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

# Using EfficientNetB0 model without top classification layer
base_model = EfficientNetB0(weights=None, include_top=False, input_tensor=inputs, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Add your custom classification layers on top of the base model
x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)

# Combine the base model with custom classification layers
model = tf.keras.Model(inputs=base_model.input, outputs=outputs)

# Load pre-trained weights
model.load_weights("/content/soybean_effecientnet.keras")

# Compile the model with appropriate optimizer, loss function, and metrics for your new dataset
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC()])

# Print model summary
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling_1 (Rescaling)     (None, 224, 224, 3)          0         ['input_2[0][0]']             
                                                                                                  
 normalization_1 (Normaliza  (None, 224, 224, 3)          7         ['rescaling_1[0][0]']         
 tion)                                                                                            
                                                                                                  
 stem_conv_pad (ZeroPadding  (None, 225, 225, 3)          0         ['normalization_1[0][0]'

In [ ]:
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

# Train the model
hist = model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=50, batch_size=32)

Epoch 1/50
4/4 [==============================] - 11s 1s/step - loss: 0.0069 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - auc: 1.0000 - val_loss: 0.0498 - val_accuracy: 0.9615 - val_precision: 0.9615 - val_recall: 0.9615 - val_auc: 0.9985
Epoch 2/50
4/4 [==============================] - 1s 228ms/step - loss: 0.0024 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - auc: 1.0000 - val_loss: 0.0417 - val_accuracy: 0.9615 - val_precision: 0.9615 - val_recall: 0.9615 - val_auc: 0.9985
Epoch 3/50
4/4 [==============================] - 1s 228ms/step - loss: 0.0209 - accuracy: 0.9902 - precision: 0.9902 - recall: 0.9902 - auc: 0.9999 - val_loss: 0.2326 - val_accuracy: 0.9615 - val_precision: 0.9615 - val_recall: 0.9615 - val_auc: 0.9601
Epoch 4/50
4/4 [==============================] - 1s 229ms/step - loss: 0.0058 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - auc: 1.0000 - val_loss: 0.2078 - val_accuracy: 0.9615 - val_precision: 0.9615 - val_recall: 0.9615 - va

### Converting Keras Model into TFLite Model

In [ ]:
tf.saved_model.save(model_, "saved_model_keras_dir")
converter = tf.lite.TFLiteConverter.from_keras_model(model_)
tflite_model = converter.convert()


In [ ]:
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

## Inception V3


In [ ]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
import numpy as np

from tensorflow.keras import layers


NUM_CLASSES = 2
IMG_SIZE = 299
size = (IMG_SIZE, IMG_SIZE)


inputs1 = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))


# Using model without transfer learning

outputs1 = InceptionV3(include_top=True, weights=None, classes=NUM_CLASSES)(inputs1)


In [ ]:
im_size = 299

images1 = []
labels1 = []

for i in dataset_path:
    data_path = path + str(i)
    filenames = [i for i in os.listdir(data_path) ]

    for f in filenames:
        img = cv2.imread(data_path + '/' + f)
        img = cv2.resize(img, (im_size, im_size))
        images1.append(img)
        labels1.append(i)

In [ ]:
images1 = np.array(images1)

images1 = images1.astype('float32') / 255.0
images1.shape

(169, 299, 299, 3)

In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


images1, Y = shuffle(images1, Y, random_state=1)


train_x, test_x, train_y, test_y = train_test_split(images1, Y, test_size=0.05, random_state=415)

#inpect the shape of the training and testing.
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(160, 299, 299, 3)
(160, 2)
(9, 299, 299, 3)
(9, 2)


In [ ]:
from keras import optimizers, losses, activations, models
model2 = tf.keras.Model(inputs1, outputs1)

model2.compile(optimizer=optimizers.SGD(lr=1e-4,
                                       momentum=0.9), loss="binary_crossentropy", metrics=["accuracy","F1Score"] )

model2.summary()

hist1 = model2.fit(train_x, train_y, epochs=50, verbose=1, callbacks=[scheduler])

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 299, 299, 3)]     0         
                                                                 
 inception_v3 (Functional)   (None, 2)                 21806882  
                                                                 
Total params: 21806882 (83.19 MB)
Trainable params: 21772450 (83.06 MB)
Non-trainable params: 34432 (134.50 KB)
_________________________________________________________________
Epoch 1/50
5/5 [==============================] - 18s 448ms/step - loss: 0.5093 - accuracy: 0.7750 - f1_score: 0.5257 - lr: 0.0100
Epoch 2/50
5/5 [==============================] - 2s 440ms/step - loss: 0.4975 - accuracy: 0.8750 - f1_score: 0.4667 - lr: 0.0100
Epoch 3/50
5/5 [==============================] - 2s 440ms/step - loss: 0.4188 - accuracy: 0.8687 - f1_score: 0.4649 - lr: 0.0100
Epoch 4/50
5/5 [==

In [ ]:
preds = model2.evaluate(test_x, test_y)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

1/1 [==============================] - 2s 2s/step - loss: 2.8736 - accuracy: 0.1111 - f1_score: 0.1000
Loss = 2.873622417449951
Test Accuracy = 0.1111111119389534


In [ ]:
hist1.history


{'loss': [0.5092523694038391,
  0.4974537491798401,
  0.41879963874816895,
  0.25515294075012207,
  0.20512254536151886,
  0.10946786403656006,
  0.06092456728219986,
  0.100009486079216,
  0.043974071741104126,
  0.030330995097756386,
  0.07662086188793182,
  0.14605741202831268,
  0.026636486873030663,
  0.04880725219845772,
  0.03109968826174736,
  0.04643229395151138,
  0.11525160074234009,
  0.010255456902086735,
  0.023270856589078903,
  0.004918561317026615,
  0.004686899483203888,
  0.009888221509754658,
  0.00923746358603239,
  0.007670666091144085,
  0.021697470918297768,
  0.00526610529050231,
  0.0026463267859071493,
  0.0018652116414159536,
  0.002742885146290064,
  0.004989479668438435,
  0.0009469389915466309,
  0.0031611707527190447,
  0.002002791967242956,
  0.0012993782293051481,
  0.0011382673401385546,
  0.0008272891864180565,
  0.0012018473353236914,
  0.0007234892691485584,
  0.002530936151742935,
  0.0010276517132297158,
  0.00045317341573536396,
  0.001053771702

Making F1 Score from scratch since the hist array is giving a combination of precision and recall

In [ ]:
f1= hist1.history['f1_score']
f1_scores = []
prec_list=[]
rec_list=[]
for pair in f1:
    precision = pair[0]
    recall = pair[1]
    f1_ = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
    f1_scores.append(f1_)
    prec_list.append(precision)
    rec_list.append(recall)

# Convert the list of F1 scores to a NumPy array
f1_scores_array = np.array(f1_scores)
prec_list= np.array(prec_list)
rec_list= np.array(rec_list)
print(f1_scores_array)
print(prec_list)
print(rec_list)

[0.30075188 0.         0.         0.75657896 0.84077329 0.84677421
 0.95285753 0.90976565 0.98527643 0.98590302 0.90025912 0.86408647
 0.98527643 0.91553133 0.98590302 0.97095589 0.91986351 1.
 0.98527643 1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.        ]
[0.86956525 0.93333334 0.92976594 0.9583333  0.96842104 0.97222215
 0.98939925 0.9785714  0.9964413  0.99641573 0.9788733  0.96774197
 0.9964413  0.9824561  0.99641573 0.99285716 0.98233217 1.
 0.9964413  1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.


Metrics

In [ ]:
import plotly.graph_objs as go
import numpy as np
epochs=list(range(1,51))

trace_accuracy = go.Scatter(x=epochs, y=hist1.history['accuracy'], mode='lines', name='Accuracy')
trace_precision = go.Scatter(x=epochs, y=prec_list, mode='lines', name='Precision')
trace_recall = go.Scatter(x=epochs, y=rec_list, mode='lines', name='Recall')
trace_f1_score = go.Scatter(x=epochs, y=f1_scores_array, mode='lines', name='F1 Score')

layout = go.Layout(
    title='Metrics Across Epochs',
    xaxis=dict(title='Epochs'),
    yaxis=dict(title='Value'),
)

# Combining traces and layout
data = [trace_accuracy, trace_precision, trace_recall, trace_f1_score]
fig = go.Figure(data=data, layout=layout)

# Displaying the plot
fig.show()